## Summarization

使用完成预训练的CodeT5对本次实验的Python代码进行注释生成

In [1]:
!pip install -q transformers
!pip install -q datasets

In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/NL-PL/

Mounted at /content/drive
/content/drive/MyDrive/NL-PL


In [3]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from inspect import getmembers, getsource, getmodule, isfunction, isroutine, isclass
import codet5, codebert

使用inspect获取代码的函数与class，

In [4]:
modules = [codet5, codebert]
functions = []
function_name = []
for module in modules:
  for member in getmembers(module, isfunction):
    name, func = member
    if name not in function_name:
      if getmodule(func).__name__ == 'codebert' or getmodule(func).__name__ == 'codet5':
        function_name.append(name)
        functions.append(getsource(func))

  for member in getmembers(module, isclass):
    name, func = member
    if name not in function_name:
      if getmodule(func).__name__ == 'codebert' or getmodule(func).__name__ == 'codet5':
        function_name.append(name)
        functions.append(getsource(func))

In [5]:
checkpoint = 'Salesforce/codet5-base-multi-sum'
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/902 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [6]:
texts = []
for func in functions:
  input_ids = tokenizer(func, return_tensors='pt', truncation=True).input_ids
  generate_ids = model.generate(input_ids)
  texts.append(tokenizer.decode(generate_ids[0], skip_special_tokens=True))

In [9]:
for i in range(len(texts)):
  print('Summarize:',texts[i])
  print('Function Name:', function_name[i])
  print('Code:',functions[i])

Summarize: Calculate the BLEU score.
Function Name: calculate_bleu
Code: def calculate_bleu(predictions, ground_truth):
  length = len(predictions)
  bleu_sum = 0
  for i in range(length):
    pred = predictions[i].split(' ')
    truth = ground_truth[i].split(' ')
    count = 0
    for word in pred:
      if word in truth:
        count += 1
    bleu_sum += count / len(pred)
  final_score = bleu_sum / length
  return final_score

Summarize: Modify data by adding missing data.
Function Name: data_modifier
Code: def data_modifier(data):
  code = data['code']
  docstring = data['docstring']

  inputs = tokenizer(code, max_length=256, padding='max_length', truncation=True)
  labels = tokenizer(docstring, max_length=128, padding='max_length', truncation=True)
  # print(labels)

  inputs['labels'] = labels['input_ids']

  return inputs

Summarize: A Beam class.
Function Name: Beam
Code: class Beam():
  def __init__(self, beam_size, cls, sep):
    # topK的K值大小
    self.size = beam_size
    # 记